In [18]:
from flask import Flask, render_template, request
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import preprocess_input
import os
from tensorflow.keras.preprocessing import image

app = Flask(__name__)
model = load_model('C:/Users/mayan/Desktop/Projects/AI ML/Alzheimer\'s Detection/final_model_01.h5')
target_img = os.path.join(os.getcwd(), 'static/images')

@app.route('/')
def index_view():
    return render_template('index.html')

# Allow files with extension png, jpg, and jpeg
ALLOWED_EXT = set(['jpg', 'jpeg', 'png'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXT

# Function to load and prepare the image in the right shape
def read_image(filename):
    img = load_img(filename, target_size=(100,100))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):  # Checking file format
            filename = file.filename
            file_path = os.path.join('static/images', filename)
            file.save(file_path)
            img = read_image(file_path)  # Preprocessing method
            class_prediction = model.predict(img)
            classes_x = np.argmax(class_prediction, axis=1)
            if classes_x == 0:
                stage = "Mild Demented"
            elif classes_x == 1:
                stage = "Moderate Demented"
            elif classes_x == 2:
                stage = "Non Demented"
            elif classes_x == 3:
                stage = "Very Mild Demented"
            else:
                stage = "Hmmmmm!!!!"
            # 'fruit', 'prob', 'user_image' these names we have seen in predict.html.
            return render_template('predict.html', stage=stage, prob=class_prediction, user_image=file_path)
        else:
            return "Unable to read the file. Please check file extension"

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, port=8000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit


1/1 [==============================] - 0s 142ms/step


127.0.0.1 - - [13/Feb/2024 23:13:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2024 23:13:44] "GET /static/images/try2.jpg HTTP/1.1" 200 -


1/1 [==============================] - 0s 31ms/step


127.0.0.1 - - [13/Feb/2024 23:14:24] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2024 23:14:24] "GET /static/images/try2.jpg HTTP/1.1" 200 -
